<a href="https://colab.research.google.com/github/Steve-Falkovsky/Hypencoder-Entity-Linking/blob/Professional-Structure/notebooks/fine_tune_Hypencoder_on_BC5CDR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import importlib.util

# for syncing changes to notebook when changing something in VScode
# deprecated/need to update to new IPython which breaks colab
# %load_ext autoreload
# %autoreload 2



REPO_NAME = "Hypencoder-Entity-Linking"
GIT_URL = f"https://github.com/Steve-Falkovsky/{REPO_NAME}.git"
BRANCH_NAME = "Professional-Structure"



# --- COLAB SETUP ---
is_colab = importlib.util.find_spec("google.colab") is not None
if is_colab:
    print("☁️ Running in Colab...")
    if not os.path.exists(REPO_NAME):
        !git clone -b {BRANCH_NAME} --single-branch {GIT_URL}

    # Move into the downloaded repo (The Root)
    os.chdir(REPO_NAME)



# --- LOCAL SETUP ---
else:
    print("💻 Running Locally...")
    if os.path.basename(os.getcwd()) == "notebooks":
        os.chdir("..")


%pip install -q -e "./hypencoder-paper"

os.chdir("./hypencoder-paper")
print(f"📍 Working Directory is now: {os.getcwd()}")
print("✅ Environment Ready!")

☁️ Running in Colab...
Cloning into 'Hypencoder-Entity-Linking'...
remote: Enumerating objects: 229, done.
remote: Counting objects: 100% (229/229), done.
remote: Compressing objects: 100% (171/171), done.
remote: Total 229 (delta 87), reused 185 (delta 51), pack-reused 0 (from 0)
Receiving objects: 100% (229/229), 656.42 KiB | 14.27 MiB/s, done.
Resolving deltas: 100% (87/87), done.
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.9/115.9 kB 5.4 MB/s eta 0:00:00
📍 Working Directory is now: /content/Hypencoder-Entity-Linking
✅ Environment Ready!


In [3]:
# loading the data
from datasets import load_dataset

dataset = load_dataset("Stevenf232/hypencoder_contrastiveLoss_nameOnly")
train_data = dataset['train']
val_data = dataset['validation']

README.md:   0%|          | 0.00/31.0 [00:00<?, ?B/s]

(…)c5cdr_train_hypencoder_contrastive.jsonl: 0.00B [00:00, ?B/s]

bc5cdr_val_hypencoder_contrastive.jsonl: 0.00B [00:00, ?B/s]

bc5cdr_test_hypencoder_contrastive.jsonl: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/2654 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2559 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2656 [00:00<?, ? examples/s]

In [4]:
# saving the data to a file
train_data.to_json('data/train.jsonl', lines=True)
val_data.to_json('data/val.jsonl', lines=True)

Creating json from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

307335

In [5]:
# tokenizing the data before training

# training
!python hypencoder_cb/utils/tokenizer_utils.py \
--standard_format_jsonl='data/train.jsonl' \
--output_file='data/train_tokenized.jsonl' \
--tokenizer="google-bert/bert-base-uncased" \
--add_special_tokens=True \
--query_max_length=32 \
--item_max_length=512

# validation
!python hypencoder_cb/utils/tokenizer_utils.py \
--standard_format_jsonl='data/val.jsonl' \
--output_file='data/val_tokenized.jsonl' \
--tokenizer="google-bert/bert-base-uncased" \
--add_special_tokens=True \
--query_max_length=32 \
--item_max_length=512

tokenizer_config.json: 100% 48.0/48.0 [00:00<00:00, 464kB/s]
config.json: 100% 570/570 [00:00<00:00, 5.73MB/s]
vocab.txt: 232kB [00:00, 26.3MB/s]
tokenizer.json: 466kB [00:00, 66.1MB/s]
2654it [00:00, 407306.36it/s]
2it [00:00, 69.79it/s]
2it [00:00, 72.84it/s]
2654it [00:00, 64031.49it/s]
2559it [00:00, 416630.07it/s]
2it [00:00, 76.63it/s]
2it [00:00, 77.55it/s]
2559it [00:00, 63742.16it/s]


Everything in the output above is [00:00] which seems quite suspicious! (or it could just be really fast)



---



# Training the hypencoder

In [7]:
!python hypencoder_cb/train/train.py hypencoder_cb/train/configs/hypencoder.2_layer_finetuned_BC5CDR.yaml

2025-12-23 00:38:19.472691: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766450299.492986   14846 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766450299.499023   14846 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766450299.515273   14846 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766450299.515297   14846 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766450299.515301   14846 computation_placer.cc:177] computation placer alr

In [8]:
%pip install huggingface_hub

# push the model to HuggingFace
# the model was saved in the model directory

from huggingface_hub import upload_folder

# Path where the model files are saved in Colab
# check and change this based on which checkpoint you got
local_folder_path = "/content/Hypencoder-Entity-Linking/hypencoder-paper/model/hypencoder.2_layer_finetuned_BC5CDR/checkpoint-80"

In [9]:


# Your desired repository ID on Hugging Face (e.g., "your-username/my-generic-model")
repo_id = "Stevenf232/hypencoder_BC5CDR"

# You may need to create the repository first if it doesn't exist
from huggingface_hub import create_repo
create_repo(repo_id, exist_ok=True)

upload_folder(
    folder_path=local_folder_path,
    repo_id=repo_id,
    repo_type="model", # or "dataset" or "space"
    commit_message="Upload trained model from Colab"
)

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...oint-80/pytorch_model.bin:   2%|1         | 8.65MB /  483MB            

  ...heckpoint-80/optimizer.pt:   1%|          | 5.56MB /  961MB            

  ...R/checkpoint-80/scaler.pt: 100%|##########| 1.38kB / 1.38kB            

  ...eckpoint-80/rng_state.pth:   7%|7         | 1.06kB / 14.7kB            

  ...heckpoint-80/scheduler.pt:   7%|7         |   105B / 1.47kB            

  ...oint-80/training_args.bin:   7%|7         |   425B / 5.91kB            

CommitInfo(commit_url='https://huggingface.co/Stevenf232/hypencoder_BC5CDR/commit/3152c4cba140185ffb93979b5121aa7bf67c8fb3', commit_message='Upload trained model from Colab', commit_description='', oid='3152c4cba140185ffb93979b5121aa7bf67c8fb3', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Stevenf232/hypencoder_BC5CDR', endpoint='https://huggingface.co', repo_type='model', repo_id='Stevenf232/hypencoder_BC5CDR'), pr_revision=None, pr_num=None)